## Primer: genomski podatki v obliki nizov znakov

Stopnja razvoja na področju biotehnologije omogoča pridobivanje bistveno več podatkov o organizmih. Eden pogostih podatkovnih tipov, s katerimi primerjamo vrste so genske zapisi. Ti so pripravni za predstavitev v računalništvu, saj jih lahko posplošimo na zaporedna štirih nukleotidov: A, C, G, T. Celoten genski zapis ki določa vse, od vaše barve oči do nagnjenosti do določenih bolezni je podano z nekaj več kot $3 \times 10^{12}$ dolgim zaporednjem DNK.

Pri razmoževanju prihaja do prepisovanja in kombiniranja DNA zapisov staršev. Ta proces seveda ni popoln, zato prihaja do napak - <i>mutacij</i>. Dolgoročna posledica mutacij pa je natanek različnih živalskih vrst, kar pomeni, da imajo sorodnejše vrste bolj podobne genske zapise.

Iz baze genskih zapisov smo naložili zaporednja mitohondrijskega gena za 13 vrst: `'Gorilla gorilla', 'Homo sapiens', 'Carassius auratus auratus', 'Delphinus capensis', 'Chamaeleo calyptratus', 'Canis lupus familiaris', 'Homo sapiens neanderthalensis', 'Rattus norvegicus', 'Equus caballus', 'Daboia russellii', 'Pan troglodytes', 'Takifugu rubripes', 'Pongo abelii', 'Sus scrofa'`.

Podatke najprej pridobimo iz spleta.

In [1]:
from Bio import Entrez
from Bio import SeqIO
import json

species = [
    ("Homo sapiens",            "NC_012920.1"),
    ("Pan troglodytes",         "NC_001643.1"),
    ("Equus caballus",          "NC_001640.1"),
    ("Chamaeleo calyptratus",   "NC_012420.1"),
    ("Delphinus capensis",      "NC_012061.1"),
    ("Takifugu rubripes",       "NC_004299.1"),
    ("Canis lupus familiaris",  "NC_002008.4"),
    ("Gorilla gorilla",         "NC_001645.1"),
    ("Pongo abelii",            "NC_002083.1"),
    ("Sus scrofa",              "NC_000845.1"),
    ("Daboia russellii",        "NC_011391.1"),
    ("Carassius auratus auratus", "NC_006580.1"),
    ("Rattus norvegicus",       "AC_000022.2"),
    ("Homo sapiens neanderthalensis", "NC_011137.1"),
]

# Data loading
infile = "podatki/seqs.json"
seqs = dict()
for name, sid in species:
    print("Loading ...", name)
    t = False
    while not t:
        try:
            handle = Entrez.efetch(db="nucleotide", rettype="gb", id=sid,
                           email="a@gmail.com")
            rec = SeqIO.read(handle, "gb")
            handle.close()
            t = True
        except:
            continue
    seqs[name] = str(rec.seq)   

json.dump(seqs, open(infile, "w"))

ModuleNotFoundError: No module named 'Bio'

In [2]:
import json
sequences = json.load(open("seqs.json"))
print(sequences["Homo sapiens"])  
print(len(sequences["Homo sapiens"]))

FileNotFoundError: [Errno 2] No such file or directory: 'seqs.json'

<font color="green"><b>Naredi sam/a.</b></font> Kako bi lahko primerjali živalske vrste glede na zapise, ki so podani kot nizi znakov? Prva ideja je, da podatke pretvorimo v vektorski prostor, v katerem bomo računali razdalje. Namig: zaporedja lahko razbiješ na manjše dele in prešteješ število pojavitev posameznih znakov, parov, trojk, ... k-terk. ahko upoštevaš tudi položaj v zaporedju. 

Dopolni in si pomagaj s funkcijo `seq_to_kmer_count`, ki pretvori niz znako v vektor števila pojavitev vseh mogožnih k-terk.

Prevedi podatke v ustrezno obliko, izvedi hierarhično gručenje in prikaži rezultate. Ali so vrste na dendrogramu postavljene smiselno? Dobiti morate sliko:

<img src="slike/nizi_dendrogram.png"></img> 

In [3]:
from itertools import product
def seq_to_kmer_count(seq, k=4):
    """
    Pretvori zaporedje seq v vektor x.
         AAAA AAAC AAAG AAAT ... TTTG TTTT
    x = [   1  1      2   10 ...   12    7]
    len(x) == len(seq) - k + 1
    """
    
    ktuples = list(zip(*[seq[i:] for i in range(k)]))     # razbijemo trenutni niz seq na k-terke
    kmers   = list(product(*(k*[["A", "C", "T", "G"]])))  # vse mozne k-terke
    
    x = np.zeros((len(kmers), ))
    ### Your code here ### 
    # for i, kmer in enumerate(kmers)
    # ...
    
    return x

In [4]:
# ...
# Za vsako zaporedje (organizem), izračunaj x
# X = np.array([x1, x2, ..., x13]) - matrika 13 x 256 (k=4)
# pozeni gručenje

Rešitev

In [5]:
from itertools import product
def seq_to_kmer_count(seq, k=4):
    ktuples = list(zip(*[seq[i:] for i in range(k)]))     # razbijemo niz na k-terke
    kmers   = list(product(*(k*[["A", "C", "T", "G"]])))  # vse mozne k-terke
    
    x = np.zeros((len(kmers), ))
    
    for ki, kmer in enumerate(kmers):
        x[ki] = ktuples.count(kmer)
    return x

# ...k = 4
keys = sequences.keys()
X    = np.zeros((len(keys), 4**k))
for ki, ky in enumerate(keys):
    seq    = sequences[ky]
    X[ki]  = seq_to_kmer_count(seq, k=k)

print(X)
print(X.shape)
H = sch.linkage(X)
D = sch.dendrogram(H, labels=list(sequences.keys()), leaf_rotation=90)
plt.ylabel("Razdalja")
plt.show()

NameError: name 'sequences' is not defined